## Q&A on a Website content through LLM
#### Retrieval augmented generation, or RAG
- Load web content using `WebBaseLoader` and `bs4`
- Create docs and divide into chunks using `RecursiveCharacterTextSplitter`
- Initialize embedding `OpenAIEmbeddings` to create vector embeddings
- Store to document chunks as vector store db (`Chroma`, `FAISS`) 
- Import and intialize LLM - `ChatOpenAI` with `gpt-4o` model
- Create prompt template
- Create chain to

**PromptTemplate** is used to dynamically generate LLM prompts by embedding variables into a predefined structure
- ChatPromptTemplate.`from_template(prompt)`
    - Creates a chat template consisting of a single message assumed to be from the human.
    - `prompt = """Suggest a restaurant name in {country} that serves {cuisine} food."""`
- ChatPromptTemplate.`from_messages(prompt)`
    - Create a chat prompt template from a variety of message formats.
    - `prompt= [("system", "Act like a doctor and answer my questions"),("user", "{input}")`

**Retriever** is an interface for fetching or retrieving relevant data from documents(chunks) or vector store

**Chain** refers to a sequence of steps or components that are linked together to perform a more complex task using a language model.
- Example - `prompt-template | llm | output-parser`
- https://python.langchain.com/v0.1/docs/modules/chains/


#### Load -> Chunk -> Embed -> Store

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from IPython.display import Markdown

loader = WebBaseLoader("https://www.sobha.com/blog/bangalore-real-estate-market-trends/")
documents = loader.load()

sp = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_chunks = sp.split_documents(documents)

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(doc_chunks, embeddings)

Markdown(db.similarity_search('Why is Bangalore rent so costly?')[0].page_content)


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\ranja\AppData\Local\Temp\ipykernel_26068\2751792405.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


7. Why is real estate booming in Bangalore? 

Real estate is booming in Bangalore due to demand driven by the city's expanding population and relatively lower expense of living. According to Bangalore real estate trends 2024, many people are looking for opulent and easily accessible dwelling options. 





8. Will flat prices go up in Bangalore 

Yes, flat prices could go up in Bangalore, as the city has witnessed the most increase in house costs, at 7%, while Mumbai saw a 6% increase. In contrast, prices in Chennai and Hyderabad increased by 5% each in the first quarter of 2023. 





9. Why is house rent so expensive in Bangalore? 

House rent is so expensive in Bangalore due to employees’ return to work-from-office, creating a spike in demand for housing. Another reason is higher commute times that have forced professionals to pick residences close to their workplaces, typically concentrated in particular locations, increasing the demand for apartments.

In [3]:
Markdown(db.similarity_search('who is the foreign minister of Bangladesh?')[0].page_content)

The Economic Impact of GIFT City on Gujarat and India
September 21st, 2024





 


Top 5 Reasons to Invest in SOBHA Infinia, Koramangala
September 20th, 2024


 








About SOBHA 

Mr P.N.C. Menon founded SOBHA Limited in 1995 with a clear vision to transform the way people perceive quality. Today, SOBHA is the most trusted brand and only backward integrated real estate player in the country.


 



Flats for Sale in Kochi
Flats in Thrissur
Flats in Kozhikode(Calicut)
Flats in Chennai
Flats in Bengaluru
Flats for Sale in Coimbatore
Flats in Delhi-NCR
Flats in GIFT City Gujarat
Flats in Pune
Flats in Mysuru
 



CAREERS
MEDIA CENTRE
SUSTAINABILITY
INVESTOR RELATIONS
SOBHA SHARE PRICE
 

GET IN TOUCH
+91 80 46464500
[email protected]
 















© Copyright 2023 All rights reserved.




Terms of Use
Privacy Policy
Disclaimer
RERA Disclaimer
Blog
Sitemap

#### Chat Prompt template

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the given context:
    <context>
    {context}
    </context>
    """
)

#### Document Chain
- *Processes a series of documents or text inputs to produce an output in stages*
- *Useful for summarization, translation, or other step-by-step transformations.*
- Stages - `Prompt Template -> LLM -> Output Parser`

In [5]:
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = ChatOpenAI(model="gpt-4o")
doc_chain = create_stuff_documents_chain(llm, prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the given context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000289C89BF460>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000289C9A393C0>, root_client=<openai.OpenAI object at 0x00000289C894E9B0>, root_async_client=<openai.AsyncOpenAI object at 0x00000289C89BF490>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': '

#### Vector Store Retriver and Retrival Chain
- *To retrieve the context information through a Retriver (here it's from vector store db)*

In [6]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
# Used to retrieve from vector store
chain = create_retrieval_chain(retriever, doc_chain)
chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000289C7D48580>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the given context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
      

#### Query

In [7]:
response = chain.invoke({"input": "Is Bangalore rent going increase?"})
Markdown(response['answer'])

Real estate is booming in Bangalore due to the city's expanding population and the relatively lower expense of living. Additionally, many people are looking for opulent and easily accessible dwelling options, driving demand in the market.

In [8]:
response = chain.invoke({"input": "Which areas are good for residential property investments?"})
Markdown(response['answer'])

What are some emerging localities in Bangalore that are experiencing substantial growth in 2024?

Emerging localities in Bangalore experiencing substantial growth in 2024 include Whitefield, Sarjapur Road, and North Bangalore. These areas are attracting considerable investment due to improved connectivity and infrastructure.

In [9]:
response = chain.invoke({"input": "Which are major infra developments coming up?"})
Markdown(response['answer'])

Based on the given context, what are some of the key infrastructure projects in Bangalore that are expected to enhance connectivity and boost real estate values?

Some of the key infrastructure projects in Bangalore expected to enhance connectivity and boost real estate values include:

1. **Bangalore Satellite Town Ring Road (STRR)** and Bangalore-Mysore Expressway, which will connect satellite cities like Devanahalli, Doddaballapur, Nelamangala, and Bidadi.
   
2. **New Metro Lines** connecting important areas and central business districts to Kempegowda International Airport, including the Blue Line metro from Silk Board to the airport, expected to be completed by mid-2026.

3. **Peripheral Ring Road** and NICE Road, which will encircle the city and significantly reduce travel time and congestion.

4. **Bangalore-Chennai Expressway** and CDP Road Network, which will further enhance connectivity and drive property values.

5. Development plans under the **Revised Master Plan 2041**, aiming to transform Bangalore into a ‘compact city’ by promoting development around public transport systems and upgrading metro and road infrastructure.

These projects will support the city’s growth, make office locations more accessible, and attract significant investment and development in emerging hotspots like Sarjapur Road, Bellandur, and Yelahanka.